In [1]:
from algosdk import account, mnemonic

private_key, address = account.generate_account()
print("My address: {}".format(address))
print("My private key: {}".format(private_key))
print("My passphrase: {}".format(mnemonic.from_private_key(private_key)))

My address: 5UHGB5GVOVBLFRGEAZMCGH2E6H6DGFQYAGFWBIC3JIQA474HPOSXLPJVWU
My private key: +usvbk8ohEvTOCCv9LCP/mpBCywPrcdIRLU8Od5wqjftDmD01XVCssTEBlgjH0Tx/DMWGAGLYKBbSiAOf4d7pQ==
My passphrase: leg sausage require lumber analyst spray shoot cage nominee sense elephant quiz lizard actress just public body bachelor follow need round order run ability stereo


In [2]:
print(f"Private key from mnemonic: {mnemonic.to_private_key('leg sausage require lumber analyst spray shoot cage nominee sense elephant quiz lizard actress just public body bachelor follow need round order run ability stereo')}")
print(f"Public key from mnemonic: {mnemonic.to_public_key('leg sausage require lumber analyst spray shoot cage nominee sense elephant quiz lizard actress just public body bachelor follow need round order run ability stereo')}")

Private key from mnemonic: +usvbk8ohEvTOCCv9LCP/mpBCywPrcdIRLU8Od5wqjftDmD01XVCssTEBlgjH0Tx/DMWGAGLYKBbSiAOf4d7pQ==
Public key from mnemonic: 5UHGB5GVOVBLFRGEAZMCGH2E6H6DGFQYAGFWBIC3JIQA474HPOSXLPJVWU


In [4]:
mnemo_2 = 'wrong afford tortoise method romance enrich car novel canoe couch start antenna imitate affair canal addict pizza talent believe gather inject wall any absent net'
priv_2 = mnemonic.to_private_key(mnemo_2)
pub_2 = mnemonic.to_public_key(mnemo_2)

In [14]:
pub_2

'FU3JH4LJUZSA23C6UKKDJ42SRRDG3YAJMEWJQJQFU4M6LXYNERORGXNBYU'

In [6]:
from algosdk.v2client import algod

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)

In [7]:
account_info = algod_client.account_info(pub_2)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Account balance: 4000120000798980 microAlgos



In [12]:
account_info = algod_client.account_info(address)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Account balance: 0 microAlgos



In [15]:
# build transaction
from algosdk.future import transaction
from algosdk import constants


params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = constants.MIN_TXN_FEE 
receiver = address
note = "Hello World".encode()
amount = 1000000
unsigned_txn = transaction.PaymentTxn(pub_2, params, receiver, amount, None, note)

In [16]:
account_info = algod_client.account_info(address)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")


Account balance: 0 microAlgos



In [17]:
# sign transaction
signed_txn = unsigned_txn.sign(priv_2)

In [18]:
txid = algod_client.send_transaction(signed_txn)

In [19]:
import json
import base64


#submit transaction

print("Successfully sent transaction with txID: {}".format(txid))

# wait for confirmation 
try:
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 4)  
except Exception as err:
    print(err)
    # return

print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))
print("Decoded note: {}".format(base64.b64decode(
    confirmed_txn["txn"]["txn"]["note"]).decode()))
print("Starting Account balance: {} microAlgos".format(account_info.get('amount')) )
print("Amount transfered: {} microAlgos".format(amount) )    
print("Fee: {} microAlgos".format(params.fee) ) 


account_info = algod_client.account_info(pub_2)
print("Final Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Successfully sent transaction with txID: QIBHI5QCWDPCUPYAH2IETWGDCR2KBZVH6UZWTVDLNYZMC7CFXZHA
Transaction information: {
    "confirmed-round": 1454,
    "pool-error": "",
    "sender-rewards": 104001039974,
    "txn": {
        "sig": "gqfxVJpvBcUq8/5qB533RmYWxtKk6lcyHmiWPXhb5W3kqw7koIRgmtrXRimvg03CCLisTepW3FYMIA//HbdECg==",
        "txn": {
            "amt": 1000000,
            "fee": 1000,
            "fv": 1371,
            "gen": "sandnet-v1",
            "gh": "7NQmXtU8mwGEzhdC2b6LD6+gq2H/aEvg4z3O14y3MIA=",
            "lv": 2371,
            "note": "SGVsbG8gV29ybGQ=",
            "rcv": "5UHGB5GVOVBLFRGEAZMCGH2E6H6DGFQYAGFWBIC3JIQA474HPOSXLPJVWU",
            "snd": "FU3JH4LJUZSA23C6UKKDJ42SRRDG3YAJMEWJQJQFU4M6LXYNERORGXNBYU",
            "type": "pay"
        }
    }
}
Decoded note: Hello World
Starting Account balance: 0 microAlgos
Amount transfered: 1000000 microAlgos
Fee: 1000 microAlgos
Final Account balance: 4000144000037974 microAlgos



In [20]:
account_info = algod_client.account_info(address)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Account balance: 1000000 microAlgos



In [21]:
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, wait_for_confirmation

In [22]:
# CREATE ASSET
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction
txn = AssetConfigTxn(
    sender=pub_2,
    sp=params,
    total=1,
    default_frozen=False,
    unit_name="LATINUM",
    asset_name="Firts NFT",
    manager=pub_2,
    reserve=pub_2,
    freeze=pub_2,
    clawback=pub_2,
    url="https://path/to/my/asset/details", 
    decimals=0)
# Sign with secret key of creator
stxn = txn.sign(priv_2)
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)
# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.
print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))
# print("Decoded note: {}".format(base64.b64decode(
#     confirmed_txn["txn"]["txn"]["note"]).decode()))


Signed transaction with txID: PX455CCQ2R2OIDLIDP3OOY6WE3KBKISSS6LFYPT7SREHN47GADAA
TXID:  PX455CCQ2R2OIDLIDP3OOY6WE3KBKISSS6LFYPT7SREHN47GADAA
Result confirmed in round: 1994
Transaction information: {
    "asset-index": 9,
    "confirmed-round": 1994,
    "pool-error": "",
    "sender-rewards": 52001872000,
    "txn": {
        "sig": "Of21GgIhrBzy1AZ/ez35OGGolp8hNOISFORds1R3gOC35VTkItkmnmxXSeUtxGO1UhDGLQJGDzi9DYVTZLS1Ag==",
        "txn": {
            "apar": {
                "an": "Firts NFT",
                "au": "https://path/to/my/asset/details",
                "c": "FU3JH4LJUZSA23C6UKKDJ42SRRDG3YAJMEWJQJQFU4M6LXYNERORGXNBYU",
                "f": "FU3JH4LJUZSA23C6UKKDJ42SRRDG3YAJMEWJQJQFU4M6LXYNERORGXNBYU",
                "m": "FU3JH4LJUZSA23C6UKKDJ42SRRDG3YAJMEWJQJQFU4M6LXYNERORGXNBYU",
                "r": "FU3JH4LJUZSA23C6UKKDJ42SRRDG3YAJMEWJQJQFU4M6LXYNERORGXNBYU",
                "t": 1,
                "un": "LATINUM"
            },
            "fee": 1000,
          